# Build anomaly 2060-2099 (2160-2199) wrt 1979-2018 onto JRA grid

## Load modules

In [1]:
import xarray as xr
import numpy  as np
import pandas as pd
from datetime import datetime
import copy as copy

## Core functions

### compute 3h anomaly (with and without leap year)

In [2]:
def compute_1m_to_3h_ano(ds_month_ano):
    """
    Purpose: compute 3h anomaly based on a monthly file
    
    Args:
        ds_month_ano : monthly anomaly [xarray dataset]
        
    Return: 
        ds_3h_ano : 3hourly anomaly [xarray dataset]
    """
    
    # Concat last 3 months at the beginning and first 3 month at the end
    # It is needed for a coorect interpolation as the end (31st december should match the 1st January minus 1 day)
    # 3 month as depending of the stencil of the interpolation scheme it could be needed
    m0=ds_month_ano.isel(month=[9,10,11])
    m0['month']=m0['month']-pd.Timedelta(days=365)
    m13=ds_month_ano.isel(month=[0,1,2])
    m13['month']=m13['month']+pd.Timedelta(days=365)
    ds_3h_ano=xr.concat([m0,ds_month_ano,m13],dim='month')

    # resampling with quadratic interpolation
    ds_3h_ano=ds_3h_ano.resample(month="3H").interpolate("quadratic")

    # select data only between 1st January to 31st december
    ds_3h_ano=ds_3h_ano.sel(month=slice('{:d}-01-01'.format(idateout), '{:d}-12-31'.format(idateout)))

    return ds_3h_ano

### Add overlap to dataset

In [3]:
def add_EW_overlap(ds):
    """
    Purpose: add 3 overlap cell on East and West boundary (useful before interpolation)
    
    Args:
        ds : input dataset [xarray dataset]
        
    Return: 
        ds : output dataset with the 3 most Eastern cells added at the West bnd 
             and the 3 most Western cells added at the East bnd [xarray dataset]
    """
    
    # Concat last 3 months at the beginning and first 3 month at the end
    # It is needed for a coorect interpolation as the end (31st december should match the 1st January minus 1 day)
    # 3 month as depending of the stencil of the interpolation scheme it could be needed
    lon0=ds.isel(longitude=slice(-3,None,1))
    lon0['longitude'] = lon0['longitude']-360.
    
    lon360=ds.isel(longitude=slice(0,3,1))
    lon360['longitude'] = lon360['longitude']+360.
    
    return xr.concat([lon0,ds_in,lon360],dim='longitude')

### Define new attributes

In [4]:
def modify_attributes_anomaly(cvar,dattrs,ds):
    """
    Purpose: modify variable reference attributes
    
    Args:
        dattrs : old attributes [dict]
        cvar   : variable name used to extract min and max [ string ]
        ds     : dataset from where min and max are extracted [ xarray dataset ]
        
    Return: 
        dattrs : new attributes [dict]
    """
    
    # get rid of non sense attributes
    if 'time' in list(dattrs.keys()):
        del dattrs["time"]
    if 'date' in list(dattrs.keys()):
        del dattrs["date"]
        
    # modify existing attributes
    dattrs["name"]      = dattrs["name"]+'_anomaly'
    dattrs["title"]     = dattrs["title"]+' anomaly'
    dattrs["long_name"] = dattrs["long_name"]+' anomaly'
    
    # compute new valid range
    dattrs["valid_min"] = ds[cvar].min().values
    dattrs["valid_max"] = ds[cvar].max().values
    
    return dattrs

## Sanity check function

### Display global mean anomaly

In [5]:
def print_annual_global_mean_ano(ds_ano,cvar):
    """
    Purpose: print annual global mean
    
    Args:
        ds_ano : dataset used to extract annual global mean [xarray dataset]
        cvar   : variable name used to extract annual global mean [ string ]
        
    Warning : grid is assumed to be regular and to scale with latitude
    """
    
    # compute annual anomaly
    zds=ds_ano[cvar].mean(dim='month')
    
    # compute weights
    weights = np.cos(np.deg2rad(zds.latitude,dtype=np.float64))
    
    # compute weighted xarray
    weights.name = "weights"
    zds = zds.weighted(weights)
    
    # display anomaly
    print('        annual mean anomaly is : ',zds.mean(("longitude", "latitude")).values)

## Parameters

In [6]:
# data dir path
cdir='DATA_in/HadCM3'

# JRA grid
cfout_grid='DATAin/JRA/JRA_grid.nc'

# list of file variables to process and the corresponding infile variables
fvarlst=['Q_1_5M','LW_TOTAL_DOWNWARD_SURFACE','P_SURF','SW_TOTAL_DOWNWARD_SURFACE','TOTAL_PRECIP','T_AIR_1_5M','U_10M','V_10M']
cvarlst =['q'     ,'ilr'                      ,'p'     ,'field203'                 ,'precip'      ,'temp'      ,'u'    ,    'v']

# reference period
yrefs=1979; yrefe=2019

# trg period
period='2060-2100' #2160-2200
if period == '2060-2100':
    # input file name extension where the data are
    cftrg_ext='2000_2099'
    # start and end of the period used
    ytrgs=2060; ytrge=2100
elif period == '2160-2200':
    cftrg_ext='2100_2199'
    ytrgs=2160; ytrge=2200
else:
    raise NameError('period not found')

cfext = '{:d}{:d}-{:d}{:d}'.format(ytrgs,ytrge,yrefs,yrefe)

idateout = 1951 # use a non leap year (0001 is out of bound for date_range pandas function )

# slice definition
slc_ref=slice('{:d}-01-01'.format(yrefs), '{:d}-01-01'.format(yrefe))
slc_trg=slice('{:d}-01-01'.format(ytrgs), '{:d}-01-01'.format(ytrge))

## Process data

In [7]:
print('Compute 3h anomaly between period {:d}-{:d} and {:d}-{:d} for data in {}'.format(ytrge,ytrgs,yrefe,yrefs,cdir))
print()

Compute 3h anomaly between period 2100-2060 and 2019-1979 for data in DATA_in/HadCM3



### Compute 3 hourly anomaly from monthly data

For each variable, the steps are :
 - open data
 - compute climato
 - compute anomaly
 - sample to 3h (quadratic interpolation)
 - write data

In [8]:
# for each variable
for ivar,fvar in enumerate(fvarlst):
    cvar=cvarlst[ivar]
    cdvar='d'+cvar
    
    print('Processing of {} ({}) in progress ...'.format(fvar,cvar))
    
    # load data reference
    print('    load reference data ...')
    cfrefin1='{}/C20_{}_1900_1999.NC'.format(cdir,fvar)
    cfrefin2='{}/A1B_{}_2000_2099.NC'.format(cdir,fvar)
    print('        file : {}'.format(cfrefin1))
    ds_ref1 = xr.open_dataset(cfrefin1)
    print('        file : {}'.format(cfrefin2))
    ds_ref2 = xr.open_dataset(cfrefin2)
    print('        combine dataset {} and {}'.format(cfrefin1,cfrefin2))
    ds_ref  = xr.combine_by_coords([ds_ref1,ds_ref2],combine_attrs='drop_conflicts').squeeze()
    
    # load data trg
    print('    load period data ...')
    cftrgin1='{}/A1B_{}_{}.NC'.format(cdir,fvar,cftrg_ext)
    print('        file : {}'.format(cftrgin1))
    ds_trg = xr.open_dataset(cftrgin1).squeeze()
    
    # drop useless variables
    if 'ht' in list(ds_ref.coords.keys()):
        ds_ref=ds_ref.drop('ht');
        ds_trg=ds_trg.drop('ht')
        
    # compute climato
    print('    build monthly climato ...')
    ds_ref_clim=ds_ref.sel(t=slc_ref).groupby('t.month').mean(dim='t')
    ds_trg_clim=ds_trg.sel(t=slc_trg).groupby('t.month').mean(dim='t')

    # compute anomaly
    print('    compute anomaly ...')
    ds_trg_ano = ds_trg_clim - ds_ref_clim

    # print global annual mean anomaly
    print_annual_global_mean_ano(ds_trg_ano,cvar)
    
    # update month dimension (use idateout as dummy date 0001 was not possible (out of bound))
    ds_trg_ano['month']=pd.date_range("{:d}/01/15".format(idateout),periods=12,freq=pd.DateOffset(months=1))
    
    # compute 3h anomaly (with and without leap year)
    ds_3h_ano=compute_1m_to_3h_ano(ds_trg_ano)
    
    # rename time dim
    ds_3h_ano=ds_3h_ano.rename({'month': 'time'})
    
    # rename variable
    ds_3h_ano=ds_3h_ano.rename({cvar:cdvar})
    
    # add variable attributes
    ds_3h_ano['d'+cvar].attrs = modify_attributes_anomaly(cdvar,ds_ref[cvar].attrs,ds_3h_ano)
    
    # add global attributes
    print('    write global att ...')
    ds_3h_ano.attrs =  { 'Description' : 'The above anomaly file is the differences between a specific period and a reference period',
                         'Specific Period' : '{:d} - {:d}'.format(ytrge, ytrgs),
                         'Reference period' : '{:d} - {:d}'.format(yrefe, yrefs),
                         'Model' : 'HadCM3',
                         'Scenario'   : 'A1B',
                         'Contact':'P. Mathiot (IGE)',
                         'Creation date':'{}'.format(datetime.now()),
                        }
    
    # write data
    print('    write data ...')
    cfout="A1B_{}_3h_ano_{}.NC".format(fvar,cfext)
    print('        file : {} ...'.format(cfout))
    ds_3h_ano.to_netcdf('DATA_out/'+cfout,unlimited_dims='time',encoding={'time' : {'dtype': 'float64'}})
    
    print('')
    print('')

Processing of Q_1_5M (q) in progress ...
    load reference data ...
        file : DATA_in/HadCM3/C20_Q_1_5M_1900_1999.NC
        file : DATA_in/HadCM3/A1B_Q_1_5M_2000_2099.NC
        combine dataset DATA_in/HadCM3/C20_Q_1_5M_1900_1999.NC and DATA_in/HadCM3/A1B_Q_1_5M_2000_2099.NC
    load period data ...
        file : DATA_in/HadCM3/A1B_Q_1_5M_2000_2099.NC
    build monthly climato ...
    compute anomaly ...
        annual mean anomaly is :  0.0014433373913731967
    write global att ...
    write data ...
        file : A1B_Q_1_5M_3h_ano_20602100-19792019.NC ...


Processing of LW_TOTAL_DOWNWARD_SURFACE (ilr) in progress ...
    load reference data ...
        file : DATA_in/HadCM3/C20_LW_TOTAL_DOWNWARD_SURFACE_1900_1999.NC
        file : DATA_in/HadCM3/A1B_LW_TOTAL_DOWNWARD_SURFACE_2000_2099.NC
        combine dataset DATA_in/HadCM3/C20_LW_TOTAL_DOWNWARD_SURFACE_1900_1999.NC and DATA_in/HadCM3/A1B_LW_TOTAL_DOWNWARD_SURFACE_2000_2099.NC
    load period data ...
        file : DATA

## Interpolate onto JRA grid

### Load JRA grid

In [9]:
cfJRA = 'DATA_in/JRA/JRA55_grid.nc'
ds_JRA = xr.open_dataset(cfJRA)
lat_JRA=ds_JRA['lat']
lon_JRA=ds_JRA['lon']

### Interpolate HadCM3 data to JRA grid

In [10]:
for ivar,fvar in enumerate(fvarlst):
    cvar=cvarlst[ivar]
    cdvar='d'+cvar
    
    # load data in
    print('    load HadCM3 data ...')
    cfin='./A1B_{}_3h_ano_{}.NC'.format(fvar,cfext)
    print('        file : {}'.format(cfin))
    ds_in = xr.open_dataset('./DATA_out/'+cfin)
    
    print('    extend input data E and W by 3 data points ...')
    ds_in = add_EW_overlap(ds_in)
    
    print('    interpolate data onto JRA grid ...')
    print('        method = cubic')
    ds_out = ds_in.interp(latitude=lat_JRA, longitude=lon_JRA, method='cubic')
    
    print('    clean output data set ...')
    print('        update valid range attributes ...')
    ds_out[cdvar].attrs['valid_min']=ds_out[cdvar].min().values
    ds_out[cdvar].attrs['valid_max']=ds_out[cdvar].max().values
    
    print('        drop useless coordinates ...')
    ds_out = ds_out.drop(['longitude','latitude'])
    
    print('        update global attributes ...')
    ds_out.attrs['Description']=ds_out.attrs['Description']+' interpolated onto JRA55 grid with a cubic method'
    ds_out.attrs['Grid']='JRA55 grid'
    ds_out.attrs['Creation date'] = '{}'.format(datetime.now())
    
    print('    write data interpolated onto JRA grid ...')
    cfout='A1B_{}_3h_ano_{}_on_JRA_grid.NC'.format(fvar,cfext)
    print('        file : ',cfout)
    ds_out.to_netcdf('./DATA_out/'+cfout)
    
    print()
    print()

    load HadCM3 data ...
        file : ./A1B_Q_1_5M_3h_ano_20602100-19792019.NC
    extend input data E and W by 3 data points ...
    interpolate data onto JRA grid ...
        method = cubic
    clean output data set ...
        update valid range attributes ...
        drop useless coordinates ...
        update global attributes ...
    write data interpolated onto JRA grid ...
        file :  A1B_Q_1_5M_3h_ano_20602100-19792019_on_JRA_grid.NC


    load HadCM3 data ...
        file : ./A1B_LW_TOTAL_DOWNWARD_SURFACE_3h_ano_20602100-19792019.NC
    extend input data E and W by 3 data points ...
    interpolate data onto JRA grid ...
        method = cubic
    clean output data set ...
        update valid range attributes ...
        drop useless coordinates ...
        update global attributes ...
    write data interpolated onto JRA grid ...
        file :  A1B_LW_TOTAL_DOWNWARD_SURFACE_3h_ano_20602100-19792019_on_JRA_grid.NC


    load HadCM3 data ...
        file : ./A1B_P_SURF